En este archivo se leeran los datos obtenidos de las caracteristicas propuestas

Se utilizará la librería de PANDAS para realizar este trabajo

In [3]:
import torch
print("GPU disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name(0))


GPU disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
import os
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

class VideoFrameDataset(Dataset):
    def __init__(self, csv_dir, selected_features=None, max_frames=None):
        """
        Conjunto de datos para clasificación de videos con longitudes variables
        
        Args:
            csv_dir (str): Directorio con archivos CSV de videos
            selected_features (list, optional): Lista de características a usar
            max_frames (int, optional): Número máximo de frames a considerar
        """
        self.videos = []
        self.labels = []
        self.video_lengths = []
        
        # Mapeo de etiquetas más flexible
        for filename in os.listdir(csv_dir):
            if filename.endswith('.csv'):
                filepath = os.path.join(csv_dir, filename)
                
                # Leer CSV con encabezados
                df = pd.read_csv(filepath)
                
                # Selección de características
                if selected_features is None:
                    # Si no se especifican, usar todas las características numéricas
                    features = df.select_dtypes(include=[np.number])
                else:
                    # Usar solo las características especificadas
                    features = df[selected_features]
                
                # Convertir a numpy array
                features_array = features.values
                
                # Limitar número de frames si se especifica
                if max_frames is not None:
                    features_array = features_array[:max_frames]
                
                # Mapeo de etiquetas basado en prefijo del nombre del archivo
                if filename.startswith('normal_'):
                    label = 0
                elif filename.startswith('sospechoso_'):
                    label = 1
                else:
                    print(f"Advertencia: Archivo {filename} ignorado - etiqueta no reconocida")
                    continue
                
                # Normalización de características del video
                scaler = StandardScaler()
                normalized_features = scaler.fit_transform(features_array)
                
                # Convertir a tensor de PyTorch
                video_tensor = torch.FloatTensor(normalized_features)
                
                self.videos.append(video_tensor)
                self.labels.append(label)
                self.video_lengths.append(len(video_tensor))
        
        # Verificar que se hayan cargado videos
        if not self.videos:
            raise ValueError("No se encontraron videos válidos. Verifica tus archivos CSV.")
        
        print(f"Total de videos cargados: {len(self.videos)}")
        print(f"Videos normales: {self.labels.count(0)}")
        print(f"Videos sospechosos: {self.labels.count(1)}")
        print(f"Longitudes de videos - Min: {min(self.video_lengths)}, Max: {max(self.video_lengths)}")
    
    def __len__(self):
        return len(self.videos)
    
    def __getitem__(self, idx):
        return self.videos[idx], torch.LongTensor([self.labels[idx]])

def collate_fn(batch):
    """
    Función personalizada para manejar lotes con longitudes variables
    """
    videos, labels = zip(*batch)
    padded_videos = pad_sequence(videos, batch_first=True)
    labels = torch.tensor(labels)
    
    return padded_videos, labels

class VideoClassificationMLP(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes, dropout_rate=0.3):
        super(VideoClassificationMLP, self).__init__()
        
        self.pool = nn.AdaptiveAvgPool1d(1)
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, num_classes))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.pool(x).squeeze(-1)
        
        return self.model(x)

def train_video_classifier(csv_dir, 
                           selected_features=None,
                           max_frames=None,
                           hidden_layers=[64, 32], 
                           learning_rate=0.001, 
                           epochs=100, 
                           batch_size=16,
                           dropout_rate=0.3):
    """
    Entrenar clasificador de videos
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Cargar datos
    dataset = VideoFrameDataset(csv_dir, selected_features, max_frames)
    
    # Dividir datos
    train_indices, val_indices = train_test_split(
        range(len(dataset)), test_size=0.2, stratify=dataset.labels, random_state=42
    )
    
    # Crear subconjuntos
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)
    
    # Crear DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    # Configurar modelo
    input_size = dataset.videos[0].shape[1]
    num_classes = 2
    
    model = VideoClassificationMLP(input_size, hidden_layers, num_classes, dropout_rate).to(device)
    
    # Configurar entrenamiento
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    best_val_accuracy = 0
    
    # Ciclo de entrenamiento
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        for batch_videos, batch_labels in train_loader:
            batch_videos = batch_videos.to(device)
            batch_labels = batch_labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_videos)
            loss = criterion(outputs, batch_labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total_predictions += batch_labels.size(0)
            correct_predictions += (predicted == batch_labels).sum().item()
        
        # Validación
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_videos, batch_labels in val_loader:
                batch_videos = batch_videos.to(device)
                batch_labels = batch_labels.to(device)
                
                outputs = model(batch_videos)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()
        
        # Calcular métricas
        train_loss_avg = train_loss / len(train_loader)
        train_accuracy = 100 * correct_predictions / total_predictions
        val_loss_avg = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        # Actualizar mejor modelo
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
    
    return best_val_accuracy

def grid_search(csv_dir, 
                hidden_layers_options=[[64, 32], [128, 64, 32], [64, 64]],
                learning_rates=[0.001, 0.0001],
                batch_sizes=[16, 32],
                dropout_rates=[0.3, 0.5]):
    """
    Realizar grid search sobre hiperparámetros
    """
    # Generar todas las combinaciones de hiperparámetros
    hyperparameter_combinations = list(itertools.product(
        hidden_layers_options, 
        learning_rates, 
        batch_sizes,
        dropout_rates
    ))
    
    # Almacenar resultados del grid search
    results = []
    
    # Iterar sobre combinaciones de hiperparámetros
    for hidden_layers, lr, batch_size, dropout_rate in hyperparameter_combinations:
        print("\nEntrenando con:")
        print(f"Capas ocultas: {hidden_layers}")
        print(f"Tasa de aprendizaje: {lr}")
        print(f"Tamaño de lote: {batch_size}")
        print(f"Tasa de dropout: {dropout_rate}")
        
        val_accuracy = train_video_classifier(
            csv_dir=csv_dir,
            hidden_layers=hidden_layers,
            learning_rate=lr,
            batch_size=batch_size,
            dropout_rate=dropout_rate,
            epochs=200,  # Reducir épocas para grid search
            max_frames=10000
        )
        
        results.append({
            'hidden_layers': hidden_layers,
            'learning_rate': lr,
            'batch_size': batch_size,
            'dropout_rate': dropout_rate,
            'val_accuracy': val_accuracy
        })
    
    # Ordenar resultados por precisión de validación
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('val_accuracy', ascending=False)
    
    # Guardar resultados
    results_df.to_csv('grid_search_results.csv', index=False)
    print("\nMejores resultados:")
    print(results_df.head())
    
    return results_df

if __name__ == '__main__':
    # Directorio con archivos CSV de videos
    csv_directory = '../datasetCSV/'
    
    # Realizar grid search
    grid_search_results = grid_search(csv_directory)


Entrenando con:
Capas ocultas: [64, 32]
Tasa de aprendizaje: 0.001
Tamaño de lote: 16
Tasa de dropout: 0.3
Total de videos cargados: 49
Videos normales: 22
Videos sospechosos: 27
Longitudes de videos - Min: 173, Max: 10000

Entrenando con:
Capas ocultas: [64, 32]
Tasa de aprendizaje: 0.001
Tamaño de lote: 16
Tasa de dropout: 0.5
Total de videos cargados: 49
Videos normales: 22
Videos sospechosos: 27
Longitudes de videos - Min: 173, Max: 10000

Entrenando con:
Capas ocultas: [64, 32]
Tasa de aprendizaje: 0.001
Tamaño de lote: 32
Tasa de dropout: 0.3
Total de videos cargados: 49
Videos normales: 22
Videos sospechosos: 27
Longitudes de videos - Min: 173, Max: 10000

Entrenando con:
Capas ocultas: [64, 32]
Tasa de aprendizaje: 0.001
Tamaño de lote: 32
Tasa de dropout: 0.5
Total de videos cargados: 49
Videos normales: 22
Videos sospechosos: 27
Longitudes de videos - Min: 173, Max: 10000

Entrenando con:
Capas ocultas: [64, 32]
Tasa de aprendizaje: 0.0001
Tamaño de lote: 16
Tasa de dropout: